In [182]:
!pip install nltk
!pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git &> /dev/null

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [183]:
import pandas as pd
import string
import nltk

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /Users/pierre/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/pierre/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pierre/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [184]:
res = []

with open("./corpus__fasttext.txt","r") as f :
    for line in f:
        label, sentence = line.split(" ",1)   
        res.append((label.replace("__label__",""), sentence))    
        
df = pd.DataFrame(res,columns=["label","text"])

df

,label,text
0,radical,"""Êtes-vous prêt à faire rendre des comptes à ..."
1,radical,"""Inapproprié [malade] pour vous ? Normal pour ..."
2,radical,"""Les termes «Cabale» et «État profond» font ré..."
3,radical,"""Le pire, c'est qu'ils adorent Satan et commet..."
4,radical,"""Symboles et logos utilisés par les pédophiles..."
...,...,...
1470,non_radical,"""Les crises économique et terroriste de 2008 e..."
1471,non_radical,"""à la différence de la crise financière de 200..."
1472,non_radical,"""Dans l’instant, la photographie de la situati..."
1473,non_radical,"""Ce recul relativement faible du revenu des mé..."


In [185]:
df_radical = df.groupby('label').get_group('radical')
df_non_radical = df.groupby('label').get_group('non_radical')

#Permet d'avoir les meme proportion pour le df_training et le df_validation
part1_rad, part2_rad = train_test_split(df_radical, test_size=0.2, random_state=13)
part1_non_rad, part2_non_rad = train_test_split(df_non_radical, test_size=0.2, random_state=13)

df_training =  pd.merge(part1_rad, part1_non_rad, how='outer')
df_validation =  pd.merge(part2_rad, part2_non_rad, how='outer')

In [186]:
french_stopwords = nltk.corpus.stopwords.words('french')
words = set(word.lower() for line in  df["text"].items() for word in line[1].split())
lemmatizer = FrenchLefffLemmatizer()
words

{'parodie',
 'longueur',
 'roriste."',
 'conformément',
 'loi',
 'l’homme',
 '(patrick',
 'd’équipe',
 'ver-',
 '"il',
 'vietnam)',
 "qu'",
 'mouches',
 'celle-ci',
 'l’idée',
 'dominateur,',
 'davantage',
 'd’inoculer',
 'musulmanisé',
 'sévir',
 'immortel',
 'considérables.',
 'guettant',
 'sociaux.',
 '(son',
 'décisions."',
 'anti-blanc"',
 'célibataire',
 'traditionnels,',
 'l’ethno-rock',
 '"vaccination',
 'muslim',
 'l’écriture',
 'asia-',
 '2022..',
 'héréditaire',
 'essais."',
 'allumés',
 'hypothèses,',
 'risible',
 'déficient.',
 'fabriqué"',
 '(17',
 'cédé',
 '"trump',
 'stoïcienne',
 '3e',
 'fin',
 'antipodes',
 'abdul',
 'téléphones',
 'amarres,',
 'éditoriale',
 'soumission',
 '«des',
 'approuve',
 'mastodon',
 'personnage',
 '«test»',
 '"heureusement',
 'attribuent',
 'bonne',
 'rédaction',
 'lorsqu’elle',
 "n'est",
 'qu’',
 'l’historicité"',
 'représentants',
 'pensait-il,',
 'répondre',
 'normal',
 'ville"",',
 'africaine',
 'veille,',
 '.....',
 'd’al-quaida',
 'coco

In [187]:
def French_Preprocess_listofSentence(listofSentence):
  preprocess_list = []
  for index, sentence in listofSentence["text"].items() :
    sentence_w_punct = "".join([i.lower() for i in sentence if i not in string.punctuation])
    sentence_w_num = ''.join(i for i in sentence_w_punct if not i.isdigit())
    tokenize_sentence = nltk.tokenize.word_tokenize(sentence_w_num, language="french")
    words_w_stopwords = [i for i in tokenize_sentence if i not in french_stopwords]
    words_lemmatize = (lemmatizer.lemmatize(w) for w in words_w_stopwords)
    sentence_clean = ' '.join(w for w in words_lemmatize if w.lower() in words or not w.isalpha())
    preprocess_list.append(sentence_clean)

  return preprocess_list

In [188]:
preprocessed_df = French_Preprocess_listofSentence(df)
preprocessed_df

['prêt faire rendre compte politique protégé',
 'malade normal sombres',
 'terme « cabale » « état profond » font référence groupe personne systématiquement infiltré gouvernement organisation monde entier obtenir position pouvoir contrôle',
 'pire adorent satan commettent contre souvent enfant',
 'symbole utilisés pédophile identifier préférence sexuelles triangle spirale amoureux triangle spirale dessiné petit amant coeur coeur amant fille coeur papillon genre non',
 'transparence seul moyen prouver public tout a vendu tant vérité rien faux récit conçu garder gens sans pouvoir garder contrôle puissance',
 'pourquoi drapeau antifa celui nazi coïncidence',
 'noir aveugle',
 'fait créés but renforcer propagande bataille empêcher vérité peuple',
 'put the full armor of so that you can take your stand against the ’',
 'donc complète dieu afin lorsque jour mal puissiez rester debout après avoir tout fait tenir debout',
 'corrompus poussés abandonner contrôle question plus fondamentales vie 

In [189]:
X_train = df_training["text"]
y_train = df_training["label"]
X_test = df_validation["text"]
y_test = df_validation["label"]

In [190]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [191]:
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)


SVC(kernel='linear')

In [192]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

 non_radical       0.64      0.36      0.46       120
     radical       0.66      0.86      0.75       175

    accuracy                           0.66       295
   macro avg       0.65      0.61      0.60       295
weighted avg       0.65      0.66      0.63       295

